## LSTM-BEV

In [ ]:
#=================== Imports ===================
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import random
import colorsys
from scipy.spatial.transform import Rotation as R
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import layers

#=================== File Paths ===================
BASE = '/Users/mahmoudalshaikh/Desktop/cadcd/C_data/cadcd/2019_02_27/'
annotations_path = "/Users/mahmoudalshaikh/Desktop/predFuture/bytetrack5.json"
lidar_folder = f"{BASE}/0031/labeled/lidar_points/data/"
image_folder = "images/byte_lstm_arrow_bytetrack5(222)"

#=================== Custom Layers ===================
class BasicBlock(layers.Layer):
    def __init__(self, filters, stride=1, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = layers.Conv2D(filters, 3, stride, padding='same', activation='relu')
        self.conv2 = layers.Conv2D(filters, 3, 1, padding='same')
        self.relu = layers.ReLU()
        self.shortcut_conv = None
        if stride != 1:
            self.shortcut_conv = layers.Conv2D(filters, 1, stride, padding='same')

    def call(self, x):
        shortcut = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.shortcut_conv:
            shortcut = self.shortcut_conv(shortcut)
        return self.relu(out + shortcut)

@tf.keras.utils.register_keras_serializable()
def mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

class CustomLSTM(tf.keras.layers.LSTM):
    @classmethod
    def from_config(cls, config):
        config.pop("time_major", None)
        return super().from_config(config)

#=================== Load Model & Normalization ===================
def load_model_and_norm(model_path, norm_path):
    model = load_model(model_path, custom_objects={"BasicBlock": BasicBlock, "mae": mse, "LSTM": CustomLSTM})
    norm = np.load(norm_path, allow_pickle=True).item()
    return model, np.array(norm['X_mean']), np.array(norm['X_std']), np.array(norm['Y_mean']), np.array(norm['Y_std'])

model_low,  X_mean_l, X_std_l, Y_mean_l, Y_std_l = load_model_and_norm("center_lstm_model_22region_right.h5", "center_lstm_norm_22region_right.npy")
model_high, X_mean_h, X_std_h, Y_mean_h, Y_std_h = load_model_and_norm("center_lstm_model_22region_left.h5", "center_lstm_norm_22region_left.npy")

#=================== Color Assignment for Cuboids ===================
def generate_random_dark_color():
    h, s, v = random.random(), random.uniform(0.6, 1), random.uniform(0.2, 0.5)
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    return '#{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255))

def generate_dark_colors_pool(n=1000):
    return list({generate_random_dark_color() for _ in range(n)})

dark_colors_pool = generate_dark_colors_pool()
cuboid_color_mapping = {}
def get_color(cuboid_id):
    if cuboid_id not in cuboid_color_mapping:
        cuboid_color_mapping[cuboid_id] = dark_colors_pool.pop() if dark_colors_pool else generate_random_dark_color()
    return cuboid_color_mapping[cuboid_id]

#=================== Track Points Over Frames ===================
def filter_points_by_previous_frame(current_points, tracked_trajectories, range_threshold, frame, cuboid_id):
    updated = []
    for traj in tracked_trajectories:
        new_traj = []
        for pt in traj:
            max_frames = 25 if pt['point'][0] > 43 else 4
            if frame - pt['frame'] < max_frames:
                new_traj.append(pt)
        if new_traj: updated.append(new_traj)

    for pt in current_points:
        added = False
        for traj in updated:
            if np.linalg.norm(np.array(traj[-1]['point']) - np.array(pt)) <= range_threshold:
                traj.append({'point': pt, 'frame': frame, 'id': cuboid_id})
                added = True
                break
        if not added:
            updated.append([{'point': pt, 'frame': frame, 'id': cuboid_id}])
    return updated

tracked_points = []

#=================== LSTM Prediction ===================
def predict_future_direction(past_pts, region):
    if len(past_pts) < 5:
        return None

    past_np = np.array(past_pts[-5:], dtype=np.float32)
    vels = np.diff(past_np, axis=0, prepend=past_np[0:1])
    enriched = np.concatenate([past_np, vels], axis=1).reshape(1, 5, 4)

    if region == "low":
        norm = (enriched - X_mean_l) / X_std_l
        pred_rel = model_low.predict(norm, verbose=0)[0]
        pred_abs = pred_rel * Y_std_l + Y_mean_l + past_np[-1]
    elif region == "high":
        norm = (enriched - X_mean_h) / X_std_h
        pred_rel = model_high.predict(norm, verbose=0)[0]
        pred_abs = pred_rel * Y_std_h + Y_mean_h + past_np[-1]
    else:
        return None

    return pred_abs[0] - past_np[-1]

past_positions = {}

#=================== Draw BEV Image ===================
def generate_bev(frame, lidar_path, annotations_path, x_range, y_range, image_folder):
    global tracked_points, past_positions

    lidar = np.fromfile(lidar_path, dtype=np.float32).reshape(-1, 4)
    x_l, y_l, z_l = lidar[:, 0], lidar[:, 1], lidar[:, 2]
    mask = (x_l > x_range[0]) & (x_l < x_range[1]) & (y_l > y_range[0]) & (y_l < y_range[1])
    x_l, y_l, z_l = x_l[mask], y_l[mask], z_l[mask]
    x_img = -y_l - y_range[0]
    y_img =  x_l - x_range[0]

    with open(annotations_path, "r") as f:
        annotations = json.load(f)

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.scatter(x_img, y_img, s=0.1, c=z_l, cmap='jet', alpha=0.9)

    current_points = []

    for cuboid in annotations[frame]["cuboids"]:
        cid = cuboid["uuid"]
        pos_x, pos_y = cuboid["position"]["x"], cuboid["position"]["y"]
        yaw = cuboid["yaw"]
        width, length = cuboid["dimensions"]["x"], cuboid["dimensions"]["y"]
        color = get_color(cid)

        # Draw box
        rot = R.from_euler('z', yaw).as_matrix()[:2, :2]
        corners = np.array([[length/2, width/2], [length/2, -width/2],
                            [-length/2, -width/2], [-length/2, width/2]])
        rotated = corners @ rot.T + [pos_x, pos_y]
        x_c, y_c = -rotated[:, 1] - y_range[0], rotated[:, 0] - x_range[0]
        ax.plot(np.append(x_c, x_c[0]), np.append(y_c, y_c[0]), color=color, lw=1.5)
        ax.scatter(-pos_y - y_range[0], pos_x - x_range[0], color=color, s=20)

        # Store position history
        if cid not in past_positions:
            past_positions[cid] = []
        past_positions[cid].append([pos_x, pos_y])
        if len(past_positions[cid]) > 7:
            past_positions[cid] = past_positions[cid][-7:]

        # Predict direction with arrow
        region = "low" if -3 <= pos_y < 7.5 else "high" if 7.5 <= pos_y < 20 else None
        vec = predict_future_direction(past_positions[cid], region) if region else None
        if vec is not None:
            start_x = -pos_y - y_range[0]
            start_y = pos_x - x_range[0]
            ax.arrow(start_x, start_y, -vec[1], vec[0],
                     color='red', head_width=1.5, head_length=2.5, lw=2, zorder=99)

        # Point logic for tracking
        if -3 <= pos_y < 7.5:
            center = [-pos_y - y_range[0], pos_x - x_range[0]]
            current_points.append({'point': center, 'id': cid})
        elif 7.5 <= pos_y <= 20:
            offsets = np.linspace(-length/2, length/2, 5)
            for offset in offsets:
                pt = np.dot(np.array([offset, 0]), rot.T) + [pos_x, pos_y]
                pt_img = [-pt[1] - y_range[0], pt[0] - x_range[0]]
                current_points.append({'point': pt_img, 'id': cid})

    filtered_pts = [p for p in current_points if 30 <= p['point'][0] <= 55]
    if frame == 0:
        tracked_points = [[{'point': pt['point'], 'frame': frame, 'id': pt['id']}] for pt in filtered_pts]
    else:
        for pt in filtered_pts:
            tracked_points = filter_points_by_previous_frame([pt['point']], tracked_points, 20, frame, pt['id'])

    for traj in tracked_points:
        for pt in traj:
            ax.scatter(pt['point'][0], pt['point'][1], color=get_color(pt['id']), s=10)

    ax.set_xlim(0, y_range[1] - y_range[0])
    ax.set_ylim(0, x_range[1] - x_range[0])
    ax.axis('off')
    ax.set_facecolor('white')

    os.makedirs(image_folder, exist_ok=True)
    plt.savefig(f"{image_folder}/bev_{frame}.png", bbox_inches='tight', pad_inches=0, dpi=200)
    plt.close(fig)

#=================== Main Loop: Generate BEV Images ===================
x_range = (-50, 50)
y_range = (-50, 50)

for frame in range(100):
    lidar_file = f"{lidar_folder}/{frame:010d}.bin"
    generate_bev(frame, lidar_file, annotations_path, x_range, y_range, image_folder)

print(f"✅ Done. Saved BEV images with past trajectory and LSTM arrows to: {image_folder}")

#=================== GIF Creation From Saved Images ===================
from PIL import Image
import re
from IPython.display import Image as IPImage, display

def natural_sort_key(string):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', string)]

def make_gif_from_images(image_folder, output_gif_file, duration=100):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    images.sort(key=natural_sort_key)
    frames = [Image.open(os.path.join(image_folder, img)) for img in images]
    frames[0].save(output_gif_file, format='GIF', append_images=frames[1:], save_all=True, duration=duration, loop=0)

image_folder = 'images/byte_lstm_arrow_bytetrack5(222)'
output_gif_path = 'byte_lstm_arrow_BT222.gif'

make_gif_from_images(image_folder, output_gif_path, duration=300)
display(IPImage(filename=output_gif_path))
